<a href="https://colab.research.google.com/github/Bryant-Dental/raptor_functions/blob/main/raptor_functions/examples/supervised_end_to_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Libraries

In [1]:
# !pip install raptor_functions

In [2]:
# !pip install mlflow optuna Boruta pycaret awscli boto3 tsfresh

### Configure AWS
- This is necessary to use train_experiments to log results and artifacts on aws instance

In [3]:
# !aws configure

### Import Packages

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import mlflow
from raptor_functions.supervised.prediction import load_model
import xgboost as xgb
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from raptor_functions.supervised.train import train_experiments
from raptor_functions.supervised.datasets import get_data
from raptor_functions.supervised.feature_extraction import get_training_features




### Load Data

In [3]:
df = get_data('handheld_data')
df.head()

,exp_unique_id,exp_name,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,Humidity (r.h.),measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,result
0,0,Test 24,1,131.938,132.443,177.444,178.157,44.974,44.508,219.374,71.286,159.292,21.965,162.449,36.144,52.439,110.970,124.839,138.676,60.717,273.972,26.905,25.456,189.092,188.708,143.288,142.050,46.07,baseline,21/3/2022,0.00,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,Control
1,0,Test 24,2,133.078,133.972,177.444,178.694,45.131,44.611,219.607,71.927,161.653,21.927,162.290,36.144,52.211,111.181,124.364,141.232,61.096,274.289,27.067,25.536,189.285,188.516,142.874,141.914,46.07,baseline,21/3/2022,0.25,14:12:54,2022-03-21 14:12:54.250,5 NegTest_3_20220321_14_12.txt,Control
2,0,Test 24,3,131.812,132.064,177.622,178.694,45.183,44.663,219.841,72.070,162.130,21.965,162.290,36.190,52.268,111.075,124.364,141.641,61.159,274.289,27.027,25.496,189.285,188.516,143.288,141.914,46.07,baseline,21/3/2022,0.50,14:12:54,2022-03-21 14:12:54.500,5 NegTest_3_20220321_14_12.txt,Control
3,0,Test 24,4,131.059,132.064,177.444,178.694,45.183,44.663,219.607,72.070,161.812,21.927,161.971,36.237,52.211,111.075,124.720,141.914,61.159,274.289,27.067,25.496,189.092,188.516,142.599,141.368,46.07,baseline,21/3/2022,0.75,14:12:54,2022-03-21 14:12:54.750,5 NegTest_3_20220321_14_12.txt,Control
4,0,Test 24,5,131.435,131.938,177.622,178.694,45.131,44.663,219.607,72.286,161.971,22.003,161.971,36.144,52.268,111.075,125.673,142.187,61.286,273.972,26.986,25.456,189.092,188.516,142.187,140.825,46.07,baseline,21/3/2022,1.00,14:12:54,2022-03-21 14:12:55.000,5 NegTest_3_20220321_14_12.txt,Control


### Feature Engineering
- Features extracted from each sensor array  for all cycles of experiment. 
- These include piecemeal features like mean, median, std, variance and transform features eg FFT and wavelength transform. 
- There is option to use only the raw signals or add the offset and gradient signals 

In [4]:
forest = RandomForestClassifier()


In [ ]:
df = get_training_features(df, offset=True, gradient=True, tree_model=forest)

In [13]:
df.head()

,exp_unique_id,exp_name,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,Humidity (r.h.),measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,exp_unique_idoffset,exp_nameoffset,timestepsoffset,sensor_1offset,sensor_2offset,sensor_3offset,sensor_4offset,sensor_5offset,sensor_6offset,sensor_7offset,sensor_8offset,sensor_9offset,sensor_10offset,sensor_11offset,sensor_12offset,sensor_13offset,sensor_14offset,sensor_15offset,sensor_16offset,sensor_17offset,sensor_18offset,sensor_19offset,sensor_20offset,sensor_21offset,sensor_22offset,sensor_23offset,sensor_24offset,Humidity (r.h.)offset,measurement_stageoffset,date_expoffset,time_elapsedoffset,datetime_exp_startoffset,datetime_expoffset,filenameoffset,exp_unique_idgradient,exp_namegradient,timestepsgradient,sensor_1gradient,sensor_2gradient,sensor_3gradient,sensor_4gradient,sensor_5gradient,sensor_6gradient,sensor_7gradient,sensor_8gradient,sensor_9gradient,sensor_10gradient,sensor_11gradient,sensor_12gradient,sensor_13gradient,sensor_14gradient,sensor_15gradient,sensor_16gradient,sensor_17gradient,sensor_18gradient,sensor_19gradient,sensor_20gradient,sensor_21gradient,sensor_22gradient,sensor_23gradient,sensor_24gradient,Humidity (r.h.)gradient,measurement_stagegradient,date_expgradient,time_elapsedgradient,datetime_exp_startgradient,datetime_expgradient,filenamegradient,result
0,0,Test 24,1,131.938,132.443,177.444,178.157,44.974,44.508,219.374,71.286,159.292,21.965,162.449,36.144,52.439,110.970,124.839,138.676,60.717,273.972,26.905,25.456,189.092,188.708,143.288,142.050,46.07,baseline,21/3/2022,0.0,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,0,Test 24,1,-0.286250,-0.366875,-0.089000,-0.447375,-0.182875,-0.135625,-0.320875,-0.892750,-2.463875,-0.009500,0.278750,-0.005875,0.149625,-0.118375,-0.134125,-3.004875,-0.474000,-0.437000,-0.111625,-0.040000,-0.048250,2.396250e-01,0.688000,0.680250,46.07,baseline,21/3/2022,0.0,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,0,Test 24,1,1.140,1.529,0.000,0.537,0.157,0.103,0.233,0.641,2.361,-0.038,-0.159,0.000,-0.228,0.211,-0.475,2.556,0.379,0.317,0.162,0.080,0.193,-0.192,-0.414,-0.136,46.07,baseline,21/3/2022,0.0,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,Control
0,1,Test 24,1,132.570,133.460,176.735,177.622,45.758,45.235,219.841,70.157,155.294,22.420,160.074,37.169,52.154,108.781,123.656,135.520,60.528,273.972,27.764,26.257,190.057,189.285,148.791,147.358,46.52,baseline,21/3/2022,0.0,14:07:01,2022-03-21 14:07:01.000,3 PosTest_2_20220321_14_07.txt,1,Test 24,1,0.426875,0.635875,-0.199125,-0.289500,-0.164750,-0.137125,-0.292500,-0.988125,-2.294625,0.033250,0.078500,0.058125,0.056625,0.000000,-0.015750,-2.912625,-0.449375,0.078875,-0.020500,-0.060500,0.048125,-2.842171e-14,1.394625,1.309625,46.52,baseline,21/3/2022,0.0,14:07:01,2022-03-21 14:07:01.000,3 PosTest_2_20220321_14_07.txt,1,Test 24,1,-0.506,-0.509,0.354,0.535,0.158,0.157,0.234,0.704,2.138,-0.076,0.000,-0.093,-0.113,0.000,0.118,2.227,0.315,0.000,0.000,0.040,0.000,0.000,-1.291,-1.136,46.52,baseline,21/3/2022,0.0,14:07:01,2022-03-21 14:07:01.000,3 PosTest_2_20220321_14_07.txt,Control
0,2,Test 24,1,133.460,134.101,176.735,177.978,45.183,44.611,218.909,71.215,158.203,22.003,162.290,36.005,52.097,110.655,124.010,138.012,60.654,273.656,27.149,25.655,188.900,188.134,142.324,140.825,46.14,baseline,21/3/2022,0.0,14:12:21,2022-03-21 14:12:21.000,5 NegTest_2_20220321_14_12.txt,2,Test 24,1,0.397500,0.320750,-0.465250,-0.179000,-0.097750,-0.143000,-0.494250,-0.883125,-2.523750,0.085250,0.219500,-0.110250,0.077500,-0.026250,0.412125,-2.750625,-0.521000,-0.237000,-0.025625,0.009625,0.096000,0.000000e+00,0.478000,0.269875,46.14,baseline,21/3/2022,0.0,14:12:21,2022-03-21 14:12:21.000,5 NegTest_2_20220321_14_12.txt,2,Test 24,1,-0.

### Model Training

In [ ]:
train_experiments(df)

### Load Model

In [ ]:

logged_model = 'fullpath/to/logged_model'
loaded_model = load_model(logged_model)


In [ ]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



In [ ]:
# Load model from pickle file.
model_pickle_filepath  = 'path/to/pickle_model'

loaded_model = load_model(logged_model)

### Prediction

In [ ]:
data = df.iloc[5,:-1].values.reshape(1,-1)

In [ ]:
prediction = loaded_model.predict(data)